In [557]:
import chipwhisperer as cw  

scope = cw.scope()
target = cw.target(scope, cw.targets.SimpleSerial) #cw.targets.SimpleSerial can be omitted
scope.default_setup()

In [559]:
import numpy as np

class IncrementalStats:
    def __init__(self):
        self.n = 0
        self.mean = None  # Start with None to handle dynamic sizes
        self.sum_sq = None
        self.variance = None

    def add_trace(self, trace):
        trace_length = len(trace)
        if self.mean is None:
            # Initialize the arrays dynamically based on the first trace length
            self.mean = np.zeros(trace_length)
            self.sum_sq = np.zeros(trace_length)
            self.variance = np.zeros(trace_length)
        elif trace_length != len(self.mean):
            # Adjust the existing arrays to accommodate a trace of new length
            # This case handles if the new trace length is shorter or longer
            if trace_length > len(self.mean):
                # Extend arrays with zeros if new trace is longer
                extension = np.zeros(trace_length - len(self.mean))
                self.mean = np.concatenate((self.mean, extension))
                self.sum_sq = np.concatenate((self.sum_sq, extension))
                self.variance = np.concatenate((self.variance, extension))
            else:
                # If new trace is shorter, use slicing to adjust
                self.mean = self.mean[:trace_length]
                self.sum_sq = self.sum_sq[:trace_length]
                self.variance = self.variance[:trace_length]

        self.n += 1
        delta = trace - self.mean
        self.mean += delta / self.n
        self.sum_sq += trace**2
        if self.n > 1:
            self.variance = (self.sum_sq - self.n * self.mean**2) / (self.n - 1)
        else:
            self.variance = np.zeros_like(self.variance)  # Variance is undefined for n=1

    def get_stats(self):
        return self.mean, self.variance


In [ ]:
def main():
    """
    Main test function to set parameters and run the trace capture and analysis.
    """

    #----------BEGIN FLAGS TO SET----------#
    max_no_traces = 1000
    total_trace_segments = 1
    scope.adc.samples = 24400
    flags = "l"
    output_directory = "traces_kyber_masked_sk"
    #----------END FLAGS TO SET------------#
    
    traces_per_seg = max_no_traces // total_trace_segments
    samples_max = scope.adc.samples
    scope.adc.offset = 0
    arr_offset = scope.adc.offset

    reset_target(scope)
    time.sleep(4)
    target.flush()
    
    flag_string = ""
    
    for f in flags:
        flag_string += f"f{f}"
    
    
    
    #Run it once to determine the trig_count
    inc_stats_trig = IncrementalStats()
    arr_0 = [[0 for i in range(samples_max)] for j in range(max_no_traces)]
    parser(f"{flag_string}", 1, arr_0, inc_stats_trig, 0)
    
    nr_runs_full_cap = (scope.adc.trig_count + samples_max - 1) // samples_max  # Round up without math lib
    print("Total trigger count for the capture: " + str(scope.adc.trig_count))
    print("Total number of runs that will be performed: " + str(nr_runs_full_cap))
    
    plt.clf()
    
    inc_stats = [0 for i in range(scope.adc.trig_count)]
     
    run_i = 0
    samples_left = scope.adc.trig_count   # Init samples_left as the total number of triggers
    
    while (run_i < nr_runs_full_cap):
        inc_stats_same_ct = IncrementalStats()
        inc_stats_random_ct = IncrementalStats()
        run_program(inc_stats_same_ct, inc_stats_random_ct, inc_stats, run_i, max_no_traces, flag_string, samples_max, samples_left, total_trace_segments, traces_per_seg, output_directory)
        samples_left -= samples_max
        run_i += 1
        
    
    target.write(b'm')
    plt.show()
    
    count = 0
    for stat in inc_stats:
        if abs(stat) > 4.5:
            count += 1
    print(f"count inc: {count}")

main()